# 3. Modelo de Machine Learning

Hemos concluido nuestro EDA, el cual nos ha  permitido entender las caracteríticas de los datos y seleccionar las variables para esta etapa que es la construcción y entrenamiento del modelo de Machine Learning, para armar un sistema de recomendación de películas. 

Se ingesta el archivo producto del EDA, el cual contine las variablescon características relevantes que permiten capturar similitudes entre ellas, a fin de implementar un sistema de recomendación:

- id: Identificador único de cada película, fundamental para enlazar y mostrar las recomendaciones.

- title: Nombre de la película, utilizado para presentar las recomendaciones de manera comprensible para el usuario.

- genres: La similitud en los géneros es clave para agrupar películas con temas similares.

- overview: Puedes generar una matriz de similitud de texto a partir de descripciones.

- actor: Pueden ayudar a mejorar la relevancia de las recomendaciones.

- director: Pueden ayudar a mejorar la relevancia de las recomendaciones.

- belongs_to_collection: Señala si la película pertenece a una franquicia o serie, lo que permite recomendaciones basadas en secuelas o temas compartidos.

Para implementar el sistema de recomendación utilizaremos el Modelo basado en la similitud de cosenos, este método es adecuado para sistemas de recomendación basados en texto, ya que permite identificar películas que tiene una relación semánticas fuerte en función de sus características textuales.
En ese sentido utilizaremos el vectorizador TF-IDF(Term Frequency-Inverse Document Frequency), el cual transforma el texto en vectores numéricos, asignando mayor peso a términos distintivos y reduciendo la influencia de palabras comunes. De esta manera, la similitud de coseno, aplicada sobre los vectores TF-IDF, nos permitirá calcular la proximidad entre películas y recomendar aquellas que sean más similares en términos de contenido y características específicas.

In [1]:
# Importamos las librerias a utilizar
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import nltk
from nltk.corpus import stopwords
#import re
#import joblib

In [2]:
# Cargar el dataset
df = pd.read_parquet('D:/2024/HenryData/Py_Individual/PI_Recomendacion/Datasets/df_model.parquet') 

In [3]:
df.head(3)

,id,title,genres,overview,actor,director,belongs_to_collection
0,862,Toy Story,"Animation, Comedy, Family","Led by Woody, Andy's toys live happily in his ...","Tom Hanks, Tim Allen, Don Rickles",John Lasseter,Toy Story Collection
1,8844,Jumanji,"Adventure, Fantasy, Family",When siblings Judy and Peter discover an encha...,"Robin Williams, Jonathan Hyde, Kirsten Dunst",Joe Johnston,No collecction
2,15602,Grumpier Old Men,"Romance, Comedy",A family wedding reignites the ancient feud be...,"Walter Matthau, Jack Lemmon, Ann-Margret",Howard Deutch,Grumpy Old Men Collection


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21631 entries, 0 to 21630
Data columns (total 7 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   id                     21631 non-null  int32 
 1   title                  21631 non-null  object
 2   genres                 21631 non-null  object
 3   overview               21631 non-null  object
 4   actor                  21631 non-null  object
 5   director               21631 non-null  object
 6   belongs_to_collection  21631 non-null  object
dtypes: int32(1), object(6)
memory usage: 1.1+ MB


In [5]:
# Inicializa el vectorizador TF-IDF
tfidf = TfidfVectorizer(stop_words='english')

# Genera la matriz de TF-IDF para 'overview'
tfidf_matrix = tfidf.fit_transform(df['overview'])

In [6]:
# Función para extraer y unificar la información de género, actor, director y coleccion
def combine_features(row):
    return row['genres'] + " " + row['actor'] + " " + row['director']+ " " + row['belongs_to_collection']

# Aplica la función de combinación de características
df['combined_features'] = df.apply(combine_features, axis=1)

In [7]:
tfidf_combined = TfidfVectorizer(max_features=5000, stop_words='english')
combined_matrix = tfidf_combined.fit_transform(df['combined_features'])

In [8]:
# Calcula la similitud del coseno sobre la matriz TF-IDF de la combinación de características
cosine_sim = cosine_similarity(combined_matrix)

In [9]:
# Guardar las matrices usando joblib
#joblib.dump(combined_matrix, 'combined_matrix.pkl')
#joblib.dump(cosine_sim, 'cosine_sim.pkl')
#joblib.dump(df, 'df.pkl')

In [10]:
# Funcion para el sistema de recomendación
def get_recommendations_1(title, cosine_sim):
    # Normaliza el título para evitar problemas con mayúsculas/minúsculas
    title = title.strip().lower()

    # Filtra el DataFrame con una comparación insensible a las mayúsculas
    matches = df[df['title'].str.lower() == title]

    # Verifica si se encontraron coincidencias
    if matches.empty:
        return f"No se encontró la película '{title.capitalize()}' en el DataFrame."

    try:
        # Obtiene el índice de la primera coincidencia
        idx = matches.index[0]

        # Calcula las puntuaciones de similitud de todas las películas
        sim_scores = list(enumerate(cosine_sim[idx]))

        # Ordena las películas según los puntajes de similitud
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

        # Obtén los índices de las 5 películas más similares (excluyendo la misma)
        top_indices = [i[0] for i in sim_scores[1:6]]

        # Retorna los títulos de las películas recomendadas
        return df['title'].iloc[top_indices].tolist()
    
    except Exception as e:
        return f"Error: {str(e)}"



In [11]:
# Llamar a la función
get_recommendations_1('Shrek', cosine_sim)

['Shrek 2',
 'Shrek Forever After',
 'Shrek the Third',
 'Shrek the Halls',
 'Scared Shrekless']

In [12]:
get_recommendations_1('Return of the Jedi', cosine_sim)

['Star Wars: The Force Awakens',
 'Star Wars: Episode I - The Phantom Menace',
 'Star Wars: Episode II - Attack of the Clones',
 'Star Wars: Episode III - Revenge of the Sith',
 'The Ewok Adventure']

In [13]:
get_recommendations_1('Titanic', cosine_sim)

['Revolutionary Road',
 'Before the Flood',
 'The Basketball Diaries',
 'Gangs of New York',
 'Hubble 3D']

In [14]:
get_recommendations_1('Wonder Woman', cosine_sim)

['Green Lantern: Emerald Knights',
 'Waitress',
 'Highway of Tears',
 'Trucker',
 'The Little Traitor']

In [15]:
get_recommendations_1('Avatar', cosine_sim)

['Avatar 2',
 'Aliens',
 'Guardians of the Galaxy',
 'Guardians of the Galaxy Vol. 2',
 'Snow White: A Tale of Terror']

In [16]:
get_recommendations_1('Batman Returns', cosine_sim)

['Batman',
 'Dark Shadows',
 'The War of the Roses',
 'Hansel and Gretel',
 'Throw Momma from the Train']

In [17]:
get_recommendations_1('Toy Story', cosine_sim)

['Toy Story 2', 'Toy Story 3', 'Luxo Jr.', 'Tin Toy', "Red's Dream"]

**Conlusión:**

El sistemas de recomendación muestsra las 5 películas más similares al título solicitado. Por ello se procede a implementar el modelo, sin embargo por un tema de memoria y rendimiento se procedará a implementar el API con una muestra representativa del 10% de la data actual.


**Muestra representativa para implementar la API**

In [18]:
# Extraer una muestra del 10% y resetear los índices
df_muestra = df.sample(frac=0.1, random_state=42).reset_index(drop=True)

In [20]:
df_muestra.head(2)

,id,title,genres,overview,actor,director,belongs_to_collection,combined_features
0,19105,Iron Maiden: Flight 666,"Documentary, Music",A chronological account of the heavy metal ban...,"Bruce Dickinson, Steve Harris, Nicko McBrain","Sam Dunn, Scot McFadyen",No collecction,"Documentary, Music Bruce Dickinson, Steve Harr..."
1,114150,Pitch Perfect,"Comedy, Music, Romance",College student Beca knows she does not want t...,"Anna Kendrick, Skylar Astin, Brittany Snow",Jason Moore,Pitch Perfect Collection,"Comedy, Music, Romance Anna Kendrick, Skylar A..."


In [19]:
df_muestra.to_parquet('D:/2024/HenryData/Py_Individual/PI_Recomendacion/Datasets/df_muestra.parquet', index=False)